## Init

In [ ]:
from dataclasses import dataclass

import numpy as np
import pandas as pd

import matplotlib.pylab as plt
import seaborn as sns

from scipy.optimize import minimize_scalar
from scipy.sparse.linalg import eigsh

from luescher_nd.hamiltonians.longrange import PhenomLRHamiltonian
from luescher_nd.zeta.zeta3d import Zeta3D
from luescher_nd.zeta.extern.pyzeta import zeta

In [ ]:
%load_ext blackcellmagic

## Set up potential parameters

In [ ]:
def gbarfix(gamma, M, mu):
    return (
        2
        * (gamma + M) ** 2
        / np.sqrt(mu * M ** 3 * (gamma ** 2 + 5 * M ** 2 + 4 * gamma * M))
        * M
    )

In [ ]:
HBARC = 197

mpi = 134 / HBARC
e0 = -2.225 / HBARC
mN = 937.326 / HBARC
mu = mN / 2

L0 = 10.0
M0 = 20*mpi
nstep = None

g0 = np.sqrt(-2 * mu * e0)
gbar0 = gbarfix(g0, M0, mu)

print(g0 / M0)

In [ ]:
def gp(p, gbar=gbar0, M=M0):
    return gbar * np.sqrt(8 * np.pi) * M**3 / (p**2 + M**2)**2

In [ ]:
n1d=30
h = PhenomLRHamiltonian(n1d=n1d, epsilon=L0/n1d, nstep=None, M=M0, gbar=gbar0)

p = np.linspace(0, np.sqrt(h.p2).max()*2, 500)

fig, ax = plt.subplots(dpi=250)

ax.plot(p, gp(p, gbar=h.gbar, M=h.M), label="Continuum", lw=1)
ax.plot(np.sqrt(h.p2), h._gp, ".", label="Discretized, $p=\sqrt{D^{\infty}(p)}$", ms=3, rasterized=True)
ax.plot(np.arange(h.n1d)*2*np.pi/h.L, h._gp[:h.n1d], "s", label=r"Discretized, $p=\frac{2 \pi}{L} n_x$", ms=1)

ax.legend(frameon=False)
ax.set_ylabel("Potential component\n" "$g(p)$ [fm]" )
ax.set_xlabel("$p$ [fm$^{-1}$]\nScattering momentum" )
ax.set_title(
    f"$N_{{1d}} = {h.n1d}$, "
    f"$\epsilon = {h.epsilon:1.2f}$ [fm], "
    f"$n_{{\mathrm{{step}}}} = \infty$,\n"
    f"$M = {h.M:1.2f}$ [fm$^{{-1}}$], "
    rf"$\bar g = {h.gbar:1.2f}$, "
    f"$\gamma = {g0:1.2f}$ [fm$^{{-1}}$]",
    size=8,
)

sns.despine()

plt.show(fig)

## Helper functions

In [ ]:
def pcotd(p, gbar, mu, M):
    d0 = 16 * gbar ** 2 * mu
    res = -(5 * gbar ** 2 * mu * M - 4 * M ** 2) / d0
    res += (15 * gbar ** 2 * mu + 16 * M) / d0 / M * p ** 2
    res += (5 * gbar ** 2 * mu + 24 * M) / d0 / M ** 3 * p ** 4
    res += (gbar ** 2 * mu + 16 * M) / d0 / M ** 5 * p ** 6
    res += 4 * M / d0 / M ** 7 * p ** 8
    return res

    

In [ ]:
@dataclass
class Kernel:
    n1d: int
    epsilon: float
    nstep: bool
    M: float
    e0: float
        
    def res(self, gbar):
        h = PhenomLRHamiltonian(n1d=self.n1d, epsilon=self.epsilon, nstep=self.nstep, M=self.M, gbar=gbar)
        ee = eigsh(h.op, which="SA", return_eigenvectors=False, k=1)
        return (self.e0 - ee.min())**2

In [ ]:
L = L0

x = np.linspace(-15, 15, 2001)
p = np.sqrt(x + 0j)/L * np.pi * 2 

fig, ax = plt.subplots(dpi=250)

ax.plot(x, zeta(x)/L/np.pi, ".", ms=1, lw=0.5, label=r"$\frac{1}{\pi L}S_3(x)$")
ax.plot(x, pcotd(p, gbar0, mu, M0).real, lw=1, label=r"$p \cot (\delta_0(p))$")

ax.legend(frameon=False)
ax.set_ylabel("Effective range expansion\n"r"$p \cot (\delta_0(p))$ [fm$^{-1}$]" )
ax.set_xlabel(r"$x = p^2 \left(\frac{L}{2\pi}\right)^2$" "\nNormalized scattering momentum" )
ax.set_title(
    f"$M = {M0:1.2f}$ [fm$^{{-1}}$], "
    rf"$\bar g = {gbar0:1.2f}$",
    size=8,
)

sns.despine()

ax.set_ylim(-5, 5)

plt.show()

## Finite Volume g

In [ ]:
data = []

for n1d in [10, 15, 20, 25]:#, 30, 35]:  # , 40, 45, 50]:
    print(n1d)
    epsilon = L0 / n1d

    kernel = Kernel(n1d, epsilon=epsilon, nstep=nstep, M=M0, e0=e0)
    res = minimize_scalar(kernel.res, bracket=(1.0e-4, 1.0e2))

    h = PhenomLRHamiltonian(n1d=n1d, epsilon=epsilon, nstep=nstep, M=M0, gbar=res.x)

    ee = eigsh(h.op, which="SA", return_eigenvectors=False, k=20)

    mu = h.m / 2
    p2 = 2 * mu * ee
    x = (h.L / 2 / np.pi) ** 2 * p2

    z = Zeta3D(L0, h.epsilon, h.nstep)(x)

    for xi, zi in zip(x, z):
        data.append(
            {
                "x": xi,
                "y": zi / np.pi / h.L,
                "epsilon": h.epsilon,
                "L": h.L,
                "n1d": n1d,
                "gbar": h.gbar,
            }
        )

df = pd.DataFrame(data).dropna().drop_duplicates()
df["y_old"] = zeta(df["x"]) / np.pi / h.L
df.head()


In [ ]:
fig, ax = plt.subplots(dpi=250)

p = np.linspace(-np.sqrt(abs(p2.min())), np.sqrt(p2.max()), 1000)
xp = (h.L / 2 / np.pi) ** 2 * p ** 2 * np.sign(p)

sns.lineplot(
    x="x",
    y="y",
    data=df.sort_values("x"),
    ax=ax,
    style="n1d",
    color="green",
    ms=5,
    lw=0.5,
    ls="--",
    markers=True,
    dashes={n1d: (2, 3) for n1d in df.n1d.unique()},
)
sns.lineplot(
    x="x",
    y="y_old",
    data=df.sort_values("x"),
    ax=ax,
    style="n1d",
    color="red",
    ms=5,
    lw=0.5,
    markers=True,
    legend=False,
    dashes={n1d: (2, 3) for n1d in df.n1d.unique()},
)
ax.plot(
    xp,
    pcotd(p * (-1j) ** (p < 0), gbar0, h.m / 2, h.M).real,
    label="Analytic",
    ls="-",
    color="blue",
    zorder=-1,
    lw=1,
)
ax.axhline(0, ls="-", color="black", zorder=-1, lw=1)

ax.plot(np.nan, np.nan, color="red", label="Original Lüscher", marker="s", ls="--")
ax.plot(np.nan, np.nan, color="green", label="Dispersion Lüscher", marker="s", ls="--")

ax.legend(fontsize=8, loc="center left", frameon=False, bbox_to_anchor=(1.0, 0.5))

ax.set_ylabel("Effective range expansion\n" r"$p \cot(\delta_0(p))$ [fm$^{-1}$]")
ax.set_xlabel(
    r"$x = p^2 \left(\frac{L}{2\pi}\right)^2$" "\nNormalized scattering momentum"
)

ax.set_title(
    "Potential fitted to reproduce infinite volume continuum ground state\n"
    f"$L = {h.L:1.2f}$ [fm], "
    f" $n_{{\mathrm{{step}}}} = \infty$,"
    f" $M = {h.M:1.2f}$ [fm$^{{-1}}$], "
    f" $\gamma = {g0:1.2f}$ [fm$^{{-1}}$],\n"
    rf" $\bar g = f(\gamma, \epsilon, L)$ fitted",
    size=8,
)


sns.despine()

plt.show(fig)


## Infinite Volume g

In [ ]:
data = []

for n1d in [10, 15, 20, 25]:#, 30, 35]:  # , 40, 45, 50]:
    print(n1d)
    epsilon = L0 / n1d

    h = PhenomLRHamiltonian(n1d=n1d, epsilon=epsilon, nstep=nstep, M=M0, gbar=gbar0)

    ee = eigsh(h.op, which="SA", return_eigenvectors=False, k=20)

    mu = h.m / 2
    p2 = 2 * mu * ee
    x = (h.L / 2 / np.pi) ** 2 * p2

    z = Zeta3D(L0, h.epsilon, h.nstep)(x)

    for xi, zi in zip(x, z):
        data.append(
            {
                "x": xi,
                "y": zi / np.pi / h.L,
                "epsilon": h.epsilon,
                "L": h.L,
                "n1d": n1d,
                "gbar": h.gbar,
            }
        )

df2 = pd.DataFrame(data).dropna().drop_duplicates()
df2["y_old"] = zeta(df2["x"]) / np.pi / h.L
df2.head()



In [ ]:
fig, ax = plt.subplots(dpi=250)

p = np.linspace(-np.sqrt(abs(p2.min())), np.sqrt(p2.max()), 1000)
xp = (h.L / 2 / np.pi) ** 2 * p ** 2 * np.sign(p)

sns.lineplot(
    x="x",
    y="y",
    data=df2.sort_values("x"),
    ax=ax,
    style="n1d",
    color="green",
    ms=5,
    lw=0.5,
    ls="--",
    markers=True,
    dashes={n1d: (2, 3) for n1d in df2.n1d.unique()}
)
sns.lineplot(
    x="x",
    y="y_old",
    data=df2.sort_values("x"),
    ax=ax,
    style="n1d",
    color="red",
    ms=5,
    lw=0.5,
    markers=True,
    legend=False,
    dashes={n1d: (2, 3) for n1d in df2.n1d.unique()}
)

ax.plot(
    xp,
    pcotd(p * (-1j) ** (p < 0), gbar0, h.m / 2, h.M).real,
    label="Analytic",
    ls="-",
    color="blue",
    zorder=-1,
    lw=1,
)
ax.axhline(
    0,
    ls="-",
    color="black",
    zorder=-1,
    lw=1,
)

ax.plot(np.nan, np.nan, color="red", label="Original Lüscher", marker="s", ls="--")
ax.plot(np.nan, np.nan, color="green", label="Dispersion Lüscher", marker="s", ls="--")

ax.legend(fontsize=8, loc="center left", frameon=False, bbox_to_anchor=(1.0, 0.5))

ax.set_ylabel("Effective range expansion\n" r"$p \cot(\delta_0(p))$ [fm$^{-1}$]")
ax.set_xlabel(
    r"$x = p^2 \left(\frac{L}{2\pi}\right)^2$"
    "\nNormalized scattering momentum"
)

ax.set_title(
    r"Potential uses infinite volume continuum parameters""\n"
    f"$L = {h.L:1.2f}$ [fm], "
    f" $n_{{\mathrm{{step}}}} = \infty$,"
    f" $M = {h.M:1.2f}$ [fm$^{{-1}}$], "
    rf" $\bar g = {h.gbar:1.2f}$, "
    "\n"
    f"$\gamma = f(\epsilon, L)$ extracted", 
    size=8
)

sns.despine()

plt.show(fig)


## Finite Volume g

In [ ]:
def pcotd_minus_zeta(x, L=L0):
    p = np.sqrt(x + 0j)/L * np.pi * 2 
    return abs(pcotd(p, gbar0, mu, M0).real - zeta(x)[0] / np.pi / L)

res = minimize_scalar(pcotd_minus_zeta, (-10, -1.e-3))

In [ ]:
lrange = np.linspace(10, 100, 100)

y = []

for L in lrange:
    res = minimize_scalar(pcotd_minus_zeta, (-1, -1.0e-3), args=(L,))

    plat2 = res.x / L ** 2 * 4 * np.pi ** 2
    elat = plat2 / 2 / mu

    y.append(elat)

y = np.array(y)

fig, ax = plt.subplots(dpi=250)

ax.plot(lrange, (e0 - y) * lrange, ".", ms=3, color="orange")

ax.set_yscale("log")

ax.set_ylabel("Difference times edge length\n" r"($E_\infty - E_L) \times L $")
ax.set_xlabel(
    r"$L$ [fm$^{-1}$]" "\nBox edge length"
)

ax.set_title(
    "Continuum but finite volume energy differences",
    size=8,
)

sns.despine()

plt.show(fig)

In [ ]:
data = []

res = minimize_scalar(pcotd_minus_zeta, (-1, -1.e-3))

plat2 = res.x / L0 ** 2 * 4 * np.pi ** 2
elat = plat2 / 2 / mu

print(elat*HBARC)


for n1d in [10, 15, 20, 25]:  # , 40, 45, 50]:
    print(n1d)
    epsilon = L0 / n1d
    
    kernel = Kernel(n1d, epsilon=epsilon, nstep=nstep, M=M0, e0=elat)
    res = minimize_scalar(kernel.res, bracket=(1.0e-4, 1.0e1))
    
    print(res.x)

    h = PhenomLRHamiltonian(n1d=n1d, epsilon=epsilon, nstep=nstep, M=M0, gbar=res.x)

    ee = eigsh(h.op, which="SA", return_eigenvectors=False, k=20)

    mu = h.m / 2
    p2 = 2 * mu * ee
    x = (h.L / 2 / np.pi) ** 2 * p2

    z = Zeta3D(L, h.epsilon, h.nstep)(x)

    for xi, zi in zip(x, z):
        data.append(
            {
                "x": xi,
                "y": zi / np.pi / h.L,
                "epsilon": h.epsilon,
                "L": h.L,
                "n1d": n1d,
                "gbar": h.gbar,
            }
        )
        
df3 = pd.DataFrame(data).dropna().drop_duplicates().query("y < 100 and y > -100")
df3["y_old"] = zeta(df3["x"])  / np.pi / L

df3.head()

In [ ]:
fig, ax = plt.subplots(dpi=250)

p = np.linspace(-np.sqrt(abs(p2.min())), np.sqrt(p2.max()), 1000)
xp = (h.L / 2 / np.pi) ** 2 * p ** 2 * np.sign(p)

sns.lineplot(
    x="x",
    y="y",
    data=df3.sort_values("x"),
    ax=ax,
    style="n1d",
    color="green",
    ms=5,
    lw=0.5,
    ls="--",
    markers=True,
    dashes={n1d: (2, 3) for n1d in df3.n1d.unique()}
)
sns.lineplot(
    x="x",
    y="y_old",
    data=df3.sort_values("x"),
    ax=ax,
    style="n1d",
    color="red",
    ms=5,
    lw=0.5,
    markers=True,
    legend=False,
    dashes={n1d: (2, 3) for n1d in df3.n1d.unique()}
)

ax.plot(
    xp,
    pcotd(p * (-1j) ** (p < 0), gbar0, h.m / 2, h.M).real,
    label="Analytic",
    ls="-",
    color="blue",
    zorder=-1,
    lw=1,
)
ax.axhline(
    0,
    ls="-",
    color="black",
    zorder=-1,
    lw=1,
)

ax.plot(np.nan, np.nan, color="red", label="Original Lüscher", marker="s", ls="--")
ax.plot(np.nan, np.nan, color="green", label="Dispersion Lüscher", marker="s", ls="--")

ax.legend(fontsize=8, loc="center left", frameon=False, bbox_to_anchor=(1.0, 0.5))

ax.set_ylabel("Effective range expansion\n" r"$p \cot(\delta_0(p))$ [fm$^{-1}$]")
ax.set_xlabel(
    r"$x = p^2 \left(\frac{L}{2\pi}\right)^2$"
    "\nNormalized scattering momentum" 
)

ax.set_title(rf"Potential fitted to finite volume Energy $E_L = {elat.real * HBARC:1.4f}$ [MeV]", size=8)

ax.set_title(
    f"Potential fitted to continuum but finite volume ground state\n"
    f"$L = {h.L:1.2f}$ [fm], "
    f" $n_{{\mathrm{{step}}}} = \infty$,"
    f" $M = {h.M:1.2f}$ [fm$^{{-1}}$], "
    f"$\gamma = \gamma_L = {np.sqrt(-2*mu*elat):1.2f}$ [fm$^{{ -1}}$]"
    "\n"
    rf" $\bar g = f(\epsilon, L, \gamma_L)$ fitted",
    size=8
)

sns.despine()

plt.show(fig)

